# WP-2 Car Adjustment

<p style="color:rgb(0,162,219); font-family:Arial; font-size:16px;">Notebook Information </p>

<table style="color:rgb(88,89,91); font-family:Arial; float:left; font-size:13px; text-align:left;">
    <tr>
        <td style="color:rgb(0,90,132);font-size:13px; text-align:left;"><b>Project</b></td>
        <td style="text-align:left;">NorMITs Demand Partner </td>
    </tr>
    <tr>
        <td style="color:rgb(0,90,132);font-size:13px; text-align:left;"><b>Primary Contact Name</b></td>
        <td style="text-align:left;"> </td>
    </tr>
    <tr>
        <td style="color:rgb(0,90,132);font-size:13px; text-align:left;"><b>Primary Contact Email</b></td>
        <td style="text-align:left;"> </td>
    </tr>
    <tr>
        <td style="color:rgb(0,90,132);font-size:13px; text-align:left;"><b>Document Sensitivity</b></td>
        <td style="text-align:left;"> </td>
    </tr>
</table>

In [1]:
import pandas as pd 
import numpy as np

In [2]:
Tour_model_output = pd.read_csv(r'I:\NTS\outputs\tour\reports\v3\matrix_county_output.csv')
Tour_model_output.head()

,tmz_o,tmz_d,mode,purpose,direction,period,trips
0,West Yorkshire,West Yorkshire,Walk,Commuting,hb_fr,1,192249.968341
1,West Yorkshire,West Yorkshire,Walk,Commuting,hb_fr,2,82663.316371
2,West Yorkshire,West Yorkshire,Walk,Commuting,hb_fr,3,30406.370034
3,West Yorkshire,West Yorkshire,Walk,Commuting,hb_fr,4,51696.947005
4,West Yorkshire,West Yorkshire,Walk,Commuting,hb_fr,5,25580.167447


In [3]:
bespoke_sectors = pd.read_excel(r'I:\NTS\WP2\Tour Model Output Analysis\Bespoke_Sectors.xlsx')
bespoke_sectors.head()

,STB,county,Region,Sector_ID,Sector,id
0,TfN,Cheshire,North West,1,Cheshire,1
1,TfN,Cleveland,North East,2,Cleveland,1
2,TfN,Cumbria,North West,3,Cumbria,1
3,TfN,Durham,North East,4,Durham,1
4,TfN,Greater Manchester,North West,5,Greater Manchester,1


In [4]:
Tour_model_output = pd.merge(Tour_model_output, bespoke_sectors, left_on= 'tmz_o' , right_on= 'county' , how= 'inner')
Tour_model_output = Tour_model_output.rename(columns={'Sector_ID':'Sector_ID_o', 'Sector':'Sector_o'}).drop(columns=['STB','county','Region','id'])
Tour_model_output = pd.merge(Tour_model_output, bespoke_sectors, left_on= 'tmz_d' , right_on= 'county' , how= 'inner')
Tour_model_output = Tour_model_output.rename(columns={'Sector_ID':'Sector_ID_d', 'Sector':'Sector_d'}).drop(columns=['STB','county','Region','id'])

1029263266.1297954
1029263266.1297954


In [5]:
Tour_model_output = Tour_model_output.groupby(['mode', 'purpose', 'direction', 'period', 'Sector_o', 'Sector_ID_o', 'Sector_d', 'Sector_ID_d']).sum('trips')

1029263266.1297954
1029263266.1297953


In [6]:
Tour_model_output=Tour_model_output.reset_index()
Tour_model_output

,mode,purpose,direction,period,Sector_o,Sector_ID_o,Sector_d,Sector_ID_d,trips
0,Air,Commuting,hb_fr,1,EEH,14,Cheshire,1,149.679401
1,Air,Commuting,hb_fr,1,Greater Manchester,5,TfSE,17,30.027875
2,Air,Commuting,hb_fr,1,London,20,Scotland,21,125.107890
3,Air,Commuting,hb_fr,1,Scotland,21,TfSE,17,45.753334
4,Air,Commuting,hb_fr,3,Midlands Connect,15,Scotland,21,22.535722
...,...,...,...,...,...,...,...,...,...
50608,Walk,Visit friends,nhb,6,Tyne and Wear,12,Tyne and Wear,12,9181.507845
50609,Walk,Visit friends,nhb,6,Wales,22,Wales,22,16680.028327
50610,Walk,Visit friends,nhb,6,Wales,22,Western Gateway,19,185.273095
50611,Walk,Visit friends,nhb,6,West Yorkshire,13,West Yorkshire,13,17677.401592


In [32]:
df = Tour_model_output.copy()

In [33]:
def calc_props(df, dirr):
    if (dirr == 'hb_fr') | (dirr == 'nhb'):
        df_total = df.groupby(['direction','mode','purpose','period','Sector_o','Sector_ID_o'])['trips'].sum().reset_index()
        df_total.rename(columns={'trips':'total_trips'}, inplace=True)
        df = pd.merge(df, df_total, on=['direction','mode','purpose','period','Sector_o','Sector_ID_o'], how='left')
        df['prop'] = df.trips / df.total_trips
    else: 
        df_total = df.groupby(['direction','mode','purpose','period','Sector_d','Sector_ID_d'])['trips'].sum().reset_index()
        df_total.rename(columns={'trips':'total_trips'}, inplace=True)
        df = pd.merge(df, df_total, on=['direction','mode','purpose','period','Sector_d','Sector_ID_d'], how='left')
        df['prop'] = df.trips / df.total_trips
    return df

result_list = []
for dirr in ['hb_fr','nhb', 'hb_to']:
    df_filter = df.loc[df['direction'] == dirr]
    df_out = calc_props(df_filter, dirr)
    result_list.append(df_out)
result_df = pd.concat(result_list, ignore_index=True)

In [10]:
df = result_df

# Create marker column to groug "Rest"
def marker(row):
    if row['direction'] in ['hb_fr', 'nhb']:
        if row['proportion'] < 0.05:
            return 'Rest'
        else:
            return row['Sector_d']
    elif row['direction'] == 'hb_to':
        if row['proportion'] < 0.05:
            return 'Rest'
        else:
            return row['Sector_o']

df['Marker'] = df.apply(marker, axis=1)
df

,mode,purpose,direction,period,Sector_o,Sector_ID_o,Sector_d,Sector_ID_d,trips,proportion,Marker
0,Air,Commuting,hb_fr,1,EEH,14,Cheshire,1,149.679401,0.000248,Rest
1,Air,Commuting,hb_fr,1,Greater Manchester,5,TfSE,17,30.027875,0.000504,Rest
2,Air,Commuting,hb_fr,1,London,20,Scotland,21,125.107890,0.000251,Rest
3,Air,Commuting,hb_fr,1,Scotland,21,TfSE,17,45.753334,0.000182,Rest
4,Air,Commuting,hb_fr,3,Midlands Connect,15,Scotland,21,22.535722,0.000158,Rest
...,...,...,...,...,...,...,...,...,...,...,...
50608,Walk,Visit friends,nhb,5,Tyne and Wear,12,Tyne and Wear,12,15299.250518,0.908485,Tyne and Wear
50609,Walk,Visit friends,nhb,5,Wales,22,Wales,22,63055.777129,0.963647,Wales
50610,Walk,Visit friends,nhb,5,West Yorkshire,13,West Yorkshire,13,15067.574421,0.945376,West Yorkshire
50611,Walk,Visit friends,nhb,5,Western Gateway,19,Peninsula Transport,16,489.246523,0.035810,Rest


In [11]:
# Filter data for direction 'hb_fr' or 'nhb' (df 1)
df1 = df[(df['direction'] == 'hb_fr') | (df['direction'] == 'nhb')]

# Filter data for direction 'hb_to' (df 2)
df2 = df[df['direction'] == 'hb_to']

# Group df 1 by 'origin' and 'marker', summing 'trips' and 'proportion'
grouped_df1 = df1.groupby(['Sector_o', 'Marker','direction','purpose', 'mode', 'period'], as_index=False).agg({
    'trips': 'sum',
    'proportion': 'sum'
      
})

# Group df 2 by 'destination' and 'marker', summing 'trips' and 'proportion'
grouped_df2 = df2.groupby(['Sector_d', 'Marker', 'direction','purpose', 'mode', 'period'], as_index=False).agg({
   'trips': 'sum',
    'proportion': 'sum'   
})



grouped_df1['Sector_d'] = grouped_df1['Marker']
grouped_df2['Sector_o'] = grouped_df2['Marker']

grouped_df1 = grouped_df1.drop(columns=['Marker'])
grouped_df2 = grouped_df2.drop(columns=['Marker'])
                                                                        

# Append grouped datasets 1 and 2
appended_dataset = pd.concat([grouped_df1, grouped_df2], ignore_index=True)



In [12]:
sector_constraint = appended_dataset
sector_constraint

,Sector_o,direction,purpose,mode,period,trips,proportion,Sector_d
0,Cheshire,hb_fr,Commuting,Bus,1,26056.515303,0.855131,Cheshire
1,Cheshire,hb_fr,Commuting,Bus,2,5582.668085,0.855131,Cheshire
2,Cheshire,hb_fr,Commuting,Bus,3,423.683085,0.855131,Cheshire
3,Cheshire,hb_fr,Commuting,Bus,4,3185.848693,0.855131,Cheshire
4,Cheshire,hb_fr,Commuting,Bus,5,2098.596900,0.855131,Cheshire
...,...,...,...,...,...,...,...,...
27019,Western Gateway,hb_to,Visit friends,Walk,2,104192.642478,0.933304,Western Gateway
27020,Western Gateway,hb_to,Visit friends,Walk,3,87322.097960,0.933304,Western Gateway
27021,Western Gateway,hb_to,Visit friends,Walk,4,86272.843587,0.933304,Western Gateway
27022,Western Gateway,hb_to,Visit friends,Walk,5,80874.631656,0.933304,Western Gateway


### Checks 

#### Check 1 - Check min and max of the proportion column. Make sure between 0 and 1.

In [13]:
min_proportion = sector_constraint['proportion'].min()
min_proportion = '{:,.6f}'.format(min_proportion)
min_proportion

'0.000008'

In [14]:
max_proportion = sector_constraint['proportion'].max()
max_proportion

0.9965589894207064

#### Check 2 - Check total trips in appended_dataset = total trips in Tour_model_output

In [22]:
total_tour_model_trips = Tour_model_output['trips'].sum()
total_sector_constraint_trips = sector_constraint['trips'].sum()

if total_tour_model_trips == total_sector_constraint_trips:
    print('Total trips match')
else:
    print('Total trips do not match')

Total trips do not match


In [16]:
total_tour_model_trips

1029263266.1297953

In [17]:
total_sector_constraint_trips

1029263266.1297954

#### Check 3 - Check the proportion column sums to 1 if you groupby:
['Sector_o', 'direction', 'purpose', 'mode', 'period'] for direction = hb_fr, nhb

['Sector_d', 'direction', 'purpose', 'mode', 'period'] for direction = hb_to

In [21]:
grouped_df1 = df1.groupby(['Sector_o','direction','purpose', 'mode', 'period'])['proportion'].sum().reset_index()
grouped_df1

,Sector_o,direction,purpose,mode,period,proportion
0,Cheshire,hb_fr,Commuting,Bus,1,0.946083
1,Cheshire,hb_fr,Commuting,Bus,2,0.946083
2,Cheshire,hb_fr,Commuting,Bus,3,0.855131
3,Cheshire,hb_fr,Commuting,Bus,4,0.859064
4,Cheshire,hb_fr,Commuting,Bus,5,0.912294
...,...,...,...,...,...,...
17122,Western Gateway,nhb,Visit friends,Walk,2,0.943009
17123,Western Gateway,nhb,Visit friends,Walk,3,0.924953
17124,Western Gateway,nhb,Visit friends,Walk,4,0.924953
17125,Western Gateway,nhb,Visit friends,Walk,5,0.960763


In [19]:
grouped_df2 = df2.groupby(['Sector_d','direction','purpose', 'mode', 'period'])['proportion'].sum().reset_index()
grouped_df2

,Sector_d,direction,purpose,mode,period,proportion
0,Cheshire,hb_to,Commuting,Bus,1,0.841557
1,Cheshire,hb_to,Commuting,Bus,2,0.942041
2,Cheshire,hb_to,Commuting,Bus,3,0.965469
3,Cheshire,hb_to,Commuting,Bus,4,0.841557
4,Cheshire,hb_to,Commuting,Bus,5,0.899521
...,...,...,...,...,...,...
5884,Western Gateway,hb_to,Visit friends,Walk,2,0.933304
5885,Western Gateway,hb_to,Visit friends,Walk,3,0.933304
5886,Western Gateway,hb_to,Visit friends,Walk,4,0.960167
5887,Western Gateway,hb_to,Visit friends,Walk,5,0.933304
